# Java 8 Features
I haven't used Java 8's various new features (lambdas, functional interfaces, etc.) for several years now. This notebook contains code used to get re-acquainted with Java 8's features.

Here are the resources I used to get myself up-to-speed:

* [Java 8 Lambdas](https://www.amazon.com/Java-Lambdas-Functional-Programming-Masses/dp/1449370772)

## Lambda Expressions
In Java, Lambda expressions implement *Functional Interfaces*. A Functional Interface is an interface that has a **single** abstract method. An functional interface should have the *@FunctionalInterface* annotation.

This one implements *Runnable*, which takes no arguments and has a *void* return type:

In [1]:
Runnable someFunc = () -> System.out.println("Hello World");

Here's one the implements the BinaryOperator interface:

In [2]:
import java.util.function.BinaryOperator;
BinaryOperator<Long> add = (x, y) -> x + y;

Here's another one that implements the Function interface:

In [3]:
import java.util.function.Function;
Function<Integer, String> toString = item -> item.toString();
String result = toString.apply(24);
System.out.println(result);

24


Here's another one that implements the Supplier interface:

In [4]:
import java.util.function.Supplier;
Supplier<Double> randomInt = () -> Math.random();
System.out.println(randomInt.get());

0.3289501331629211


## Using Lambda Expressions in Methods
You can use a Lambda expression as a method parameter:

In [5]:
void runSomethingWithLog(Runnable funcToRun) {
    System.out.println("Before");
    funcToRun.run();
    System.out.println("After\n");
}

runSomethingWithLog(() -> System.out.println("Doing something!"));
runSomethingWithLog(() -> System.out.println("Doing something else!"));

Before
Doing something!
After

Before
Doing something else!
After



## Streams
The *streams* API in Java 8 provides high-level collections-processing libraries. They make heavy use of lambda expressions.\

You can use the streams API in cases where you would explicitly iterate over collections:

In [6]:
import java.util.Arrays;
List<Integer> examples = Arrays.asList(1, 2, 3, 4, 5);

// Explicit iteration
long countGreaterThan3 = 0;
for(Integer example : examples) {
    if(example > 3) {
        countGreaterThan3++;
    }
}
System.out.println(countGreaterThan3);

// Streams API
countGreaterThan3 = examples.stream()
    .filter(item -> item > 3)
    .count();
System.out.println(countGreaterThan3);

2
2


In the streams API, the final *terminal* actions are the only things that are eagerly evaluated. Everything else (those methods that return another stream) is lazily evaluated. This doesn't execute *filter*, for example, because it is missing one of the terminal actions:

In [7]:
List<Integer> examples = Arrays.asList(1, 2, 3, 4, 5);
examples.stream()
    .filter(item -> {
        System.out.println("This shouldn't print");
        return item > 3;
    })

java.util.stream.ReferencePipeline$2@cc11975

The *Collectors* methods are used to take a stream and transform it back into a collection:

In [8]:
import static java.util.stream.Collectors.*;
List<Integer> examples = Arrays.asList(1, 2, 3);
List<String> sillyTransform = examples.stream()
    .map(item -> "Some string")
    .collect(toList());
System.out.println(sillyTransform);

[Some string, Some string, Some string]


## Default Methods
Default methods are a powerful way to add new API features to existing classes without breaking backwards-compatibility. For example, the Collections APIs were enhanced with tons of new functionality in Java 8 through default methods.

The following is an example of using a default method:

In [9]:
interface SomeParent {
    void printSomething(String body);
    
    default void usePrintSomething() {
        printSomething("Hello from default method");
    }
}

class SomeChild implements SomeParent {
    public void printSomething(String body) {
        System.out.println("printSomething body: " + body);
    }
}


SomeParent theObject = new SomeChild();
theObject.usePrintSomething();

printSomething body: Hello from default method


## Method References
Method references are a bit of syntactic sugar that simplify this common pattern:

In [10]:
List<String> list = Arrays.asList("we", "are", "lowercase");
List<String> upperCased = list.stream()
    .map(item -> item.toUpperCase())
    .collect(toList());
System.out.println(upperCased);

[WE, ARE, LOWERCASE]


The above example uses `.map(item -> item.toString())`. When you have a lambda that just calls a single method you can simplify it using method references:

In [11]:
List<String> upperCased = list.stream()
    .map(String::toUpperCase) // The method reference syntax is classname::methodname
    .collect(toList());
System.out.println(upperCased);

[WE, ARE, LOWERCASE]


## Collectors
Java has the java.util.streams.Collectors class that has a lot of useful helper methods to transform streams into various output data structures. Most of these are effectively what you would have written in a `reduce()` call, just at a nice higher level.

I've already used the toList() collector method. Here's another example using partitioningBy()

In [12]:
List<Integer> someNumbers = Arrays.asList(1,2,3,4,5);
Map<Boolean, List<Integer>> partitioned = someNumbers.stream()
    .map(item -> item + 1)
    .collect(partitioningBy(item -> item < 4));
System.out.println(partitioned);

{false=[4, 5, 6], true=[2, 3]}


Here's another example using grouping by:

In [13]:
List<String> someWords = Arrays.asList("these", "are", "some", "terrific", "words");
Map<Character, List<String>> groupedBy = someWords.stream()
    .collect(groupingBy(item -> item.charAt(0)));
System.out.println(groupedBy);

{a=[are], s=[some], t=[these, terrific], w=[words]}
